# Week 3 Section 1

## Import Section

In [2]:
#Install BeautifulSoup Package
! pip install beautifulsoup4
! easy_install lxml
print("Installation successful")

Searching for lxml
Best match: lxml 4.5.0
Processing lxml-4.5.0-py3.6-linux-x86_64.egg
lxml 4.5.0 is already the active version in easy-install.pth

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/lxml-4.5.0-py3.6-linux-x86_64.egg
Processing dependencies for lxml
Finished processing dependencies for lxml
Installation successful


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Import operation is successful!")

Import operation is successful!


## Scraping Section to obtain dataframe

In [4]:
# We will scrape the wikipedia

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extraction = requests.get(url).text
wiki_data = BeautifulSoup(extraction, features='html')

In [5]:
column_names = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postalcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postalcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df_toronto = df_toronto.append({'Postal Code': postalcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#Copy the data from original df_toronto
df_toronto_original = df_toronto
df_toronto = df_toronto_original.copy()

## Dataframe Cleaning Section

In [6]:
#Cleaning the table
df_toronto = df_toronto[df_toronto.Borough!='Not assigned']
df_toronto = df_toronto[df_toronto.Borough!= 0]
df_toronto.reset_index(drop = True, inplace = True)

#Assigning borough name to neighborhood name if the neighborhood name is 'Not Assigned'
df_toronto.loc[df_toronto['Neighborhood']=='Not assigned', 'Neighborhood'] \
= df_toronto.loc[df_toronto['Neighborhood']=='Not assigned', 'Borough']

#Aggregating Neighborhoods with the same Postalcode
df_final = df_toronto.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_final.head(15)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
#Last Part for Section 1
print("Rows = {}".format(df_final.shape[0]))

Rows = 103


# Start of Section 2
## Accessing coordinate data from CSV file

In [8]:
#Reading coordinate data from downloaded file
coordinate_data = pd.read_csv('Geospatial_Coordinates.csv')
coordinate_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging coordinate data to original table

In [9]:
df_merged = df_final.merge(coordinate_data, on='Postal Code')
df_merged.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
#Saving the merged table to csv
df_merged.to_csv('Merged_Data.csv')

# End of Section 2
## Thank you very much.